# Parallel multi-point sequential Bayesian Optimisation

In [1]:
import torch
from nubo.acquisition import MCExpectedImprovement, MCUpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import sequential
from nubo.test_functions import Hartmann6D
from nubo.utils import LatinHypercubeSampling, unnormalise
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
torch.manual_seed(1)
lhs = LatinHypercubeSampling(dims=dims)
x_train = lhs.maximin(points=dims*5)
x_train = unnormalise(x_train, bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 10

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    mll = ExactMarginalLogLikelihood(likelihood=likelihood, model=gp)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, mll=mll, lr=0.1, steps=200)

    # specify acquisition function
    # acq = MCExpectedImprovement(gp=gp, y_best=torch.max(y_train), samples=256)
    acq = MCUpperConfidenceBound(gp=gp, beta=1.96**2, samples=256)

    # optimise acquisition function
    x_new, _ = sequential(func=acq, method="Adam", batch_size=4, bounds=bounds, lr=0.1, steps=200, num_starts=1)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if torch.max(y_new) > torch.max(y_train[:-y_new.size(0)]):
        best_eval = torch.argmax(y_train)
        print(f"New best at evaluation {best_eval+1}: \t Inputs: {x_train[best_eval, :].numpy().reshape(dims).round(4)}, \t Outputs: {-y_train[best_eval].numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 31: 	 Inputs: [4.221e-01 9.083e-01 8.117e-01 4.429e-01 3.282e-01 8.000e-04], 	 Outputs: -2.6071
New best at evaluation 35: 	 Inputs: [0.4154 0.9929 0.992  0.5121 0.0048 0.0012], 	 Outputs: -2.7314
New best at evaluation 43: 	 Inputs: [0.4165 0.915  0.9966 0.4999 0.0085 0.073 ], 	 Outputs: -2.9413
New best at evaluation 47: 	 Inputs: [0.4151 0.8767 0.9961 0.5564 0.0012 0.0177], 	 Outputs: -3.1591
New best at evaluation 67: 	 Inputs: [0.4031 0.8638 0.9958 0.5584 0.001  0.0313], 	 Outputs: -3.1753
Evaluation: 67 	 Solution: 3.1753
